In [ ]:
# ! pip install progressbar

In [1]:
"""

extraction_pipeline.py

by Jeff, Manu and Tanc

"""

import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import boto3
# print(tf.__version__)
import ingestor
import extractor2
import data
import functools
import progressbar

# config
BUCKET_NAME = 'sagemaker-cs281'
data_paths = {
#     'train':'deephol-data-processed/proofs/human/train_new',
    'valid':'deephol-data-processed/proofs/human/valid_new',
    'test':'deephol-data-processed/proofs/human/test_new'
}

Tesor Flow Version: 1.14.0 Ingest File
Tesor Flow Version: 1.14.0  Utility File
Tesor Flow Version: 1.14.0 Extactor 2


In [2]:
data_split ='valid'

# check
import inspect
print(inspect.getsource(data.get_train_dataset))

def get_train_dataset(params):
  path = os.path.join(params.dataset_dir, 'test') # 'train*')
  files =  tf.io.gfile.listdir(path)
  files = [os.path.join(path, f) for f in files if 'pbtxt' not in f]
#   return tf.data.TFRecordDataset(files)
  if not files:
    raise ValueError('No training files found in %s' % path)
  return tfrecord_dataset_with_source(files, SOURCE_DATASETDIR)



# Functions

In [3]:
def upload_np_to_s3(array, object_name):    
    # save localy
    local_filename = '/tmp/temp.csv'
    
    np.savetxt(local_filename, array, delimiter=',')
    
    # s3 upload
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(local_filename, BUCKET_NAME, object_name)

# Run

In [4]:

# make tf dataset of parsed examples
params = ingestor.get_params()
train_data = data.get_train_dataset(params)
parser = data.tristan_parser
train_parsed = train_data.map(functools.partial(parser, params=params))

# set features and labels
features = {'goal': [], 'goal_asl': [], 'thms': [], 'thms_hard_negatives': []}
labels = {'tac_id': []}

# iterate over dataset to extract data into arrays
train_parsed = train_parsed
for raw_record in train_parsed:
    fx, lx = raw_record[0], raw_record[1]
    features['goal'].append(fx['goal'])
    features['goal_asl'].append(fx['goal_asl'])
    features['thms'].append(fx['thms'])
    features['thms_hard_negatives'].append(fx['thms_hard_negatives'])
    labels['tac_id'].append(lx['tac_id'])

In [5]:
# instantiate extractor object
ex = extractor2.Extractor(params)

# tokenize goals
features['goal_ids'] = ex.tokenize(features['goal'], ex.vocab_table)

# tokenize hypotheses
length = len(features['goal'])
features['goal_asl_ids'] = []
for i in range(length):
    temp = ex.tokenize(features['goal_asl'][i], ex.vocab_table)
    features['goal_asl_ids'].append(temp)


Instructions for updating:
Use `tf.cast` instead.


In [6]:
# free memory
del features['goal']
del features['goal_asl']
del features['thms']
del features['thms_hard_negatives']

In [7]:
# features['goal_ids'] is now an array of size N x 1000
features['goal_ids'] = features['goal_ids'].numpy()
print('Number of training examples:', len(features['goal_ids']))
print('Size of training examples:', len(features['goal_ids'][0]))

# features['goal_asl_ids'] is now an array of size  N x ? x 1000
length = len(features['goal_asl_ids'])
for i in range(length):
    features['goal_asl_ids'][i] = [hypothesis.numpy() 
                                   for hypothesis in features['goal_asl_ids'][i]]
print('Number of training examples:', len(features['goal_asl_ids']))
print('Number of hypotheses for an example:', len(features['goal_asl_ids'][0]))

Number of training examples: 122928
Size of training examples: 1000
Number of training examples: 122928
Number of hypotheses for an example: 2


In [8]:
# features['tactic_ids'] is now an array of size N x 1
labels['tac_id'] = [i.numpy() for i in labels['tac_id']]
print('Number of training examples:', len(labels['tac_id']))

Number of training examples: 122928


In [9]:

# convert goals to numpy arrays
goals = np.array(features['goal_ids'])
print(np.shape(goals))

(122928, 1000)


In [10]:
# convert goal hypotheses to numpy arrays and concatenate
hypotheses = features['goal_asl_ids']
length_hyp = len(hypotheses)
for i in range(length_hyp):
    if (len(hypotheses[i]) != 0):
        hypotheses[i] = np.concatenate(hypotheses[i])  # concatenate hypotheses in a given hypothesis list
        hypotheses[i] = hypotheses[i][hypotheses[i] != 0]  # remove zeroes in between
        hypotheses[i] = hypotheses[i][0:3000]  # truncate to max hyp length = 3000 chars (< than 10% of data
        len_conc = len(hypotheses[i]) # pad with zeroes to make length 3000 (to save as csv)
        hypotheses[i] = np.pad(hypotheses[i], (0, 3000-len_conc), mode='constant')
    else:
        hypotheses[i] = np.zeros(3000, dtype = 'int32')
np.set_printoptions(threshold=np.sys.maxsize)
print(np.shape(hypotheses))


(122928, 3000)


In [11]:
# convert tactics to numpy arrays and one-hot encode
a = np.array(labels['tac_id'])
tactics = np.zeros((a.size, 40+1))
tactics[np.arange(a.size),a] = 1
print(np.shape(tactics))

X_train, Y_train = goals, tactics
print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(hypotheses))

(122928, 41)
(122928, 1000)
(122928, 41)
(122928, 3000)


In [12]:
# create feature matrix with goals and hypotheses
length = len(X_train)
X_train_hyp = []
for i in range(length):
    train_example = np.concatenate((X_train[i], hypotheses[i]))  # concatenate goal and hypotheses
    train_example = train_example[train_example != 0]  # remove zeroes in between
    train_example = train_example[0:3000]  # truncate to max hyp length of 3000 chars (less than 10% of data
    len_conc = len(train_example)  # pad with zeroes to make length 3000 (to save as csv)
    train_example = np.pad(train_example, (0, 3000-len_conc), mode='constant')
    X_train_hyp.append(np.asarray(train_example, dtype='float64').tolist())
X_train_hyp = np.array(X_train_hyp)
print(np.shape(X_train_hyp))    

(122928, 3000)


In [13]:
# # save to s3
# partition_size = PARTITION_SIZE if len(Y_train) > PARTITION_SIZE else len(Y_train) 
# n_partitions = len(Y_train) // partition_size
# print(len(Y_train), partition_size, n_partitions)

# for i, split in enumerate(np.array_split(X_train, n_partitions), 1):
#     upload_np_to_s3(split, os.path.join(data_paths[data_split], 'X_train_{}.csv'.format(i)))
# print('Uploaded all X_train files')
b
# for i, split in enumerate(np.array_split(X_train_hyp, n_partitions), 1):
#     upload_np_to_s3(split, os.path.join(data_paths[data_split], 'X_train_hyp_{}.csv'.format(i)))
# print('Uploaded all X_train_hyp files')

# upload_np_to_s3(Y_train, os.path.join(data_paths[data_split], 'Y_train.csv'))
# print('Uploaded Y_train file')


In [14]:
# NEED TO SPLIT ARRAYS THE RIGHT WAY
print([x.shape for x in [X_train, X_train_hyp, Y_train]])

[(122928, 1000), (122928, 3000), (122928, 41)]


In [47]:
splits = split_array_batches(X_train)

skipping 1654 examples to have equal length batches
partition size: 4096
shape of truncated array:  (102400, 1000)


In [49]:
print([x.shape for x in splits])

[(4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000)]


## Split arrays

In [16]:
def split_array_batches(array):
    """ split in equal length array """
    n_examples = len(array)
    partition_size = 4096
    end_to_skip = n_examples % partition_size  # arbitrary
    print('skipping {} examples to have equal length batches'.format(end_to_skip))
    n_partitions = (n_examples - end_to_skip) / partition_size 

    print('partition size: {}'.format(partition_size))
    print('shape of truncated array: ', array[:-end_to_skip].shape)

    splits = np.split(array[:-end_to_skip], n_partitions)
    
    return splits

# Read and re split

In [16]:
# import pandas as pd
# import generator
# import importlib

# importlib.reload(generator)

<module 'generator' from '/home/ec2-user/SageMaker/deepmath/deepmath/deephol/train/generator.py'>

In [17]:
data_paths = {'train':'deephol-data-processed/proofs/human/train',
            'valid':'deephol-data-processed/proofs/human/valid',
            'test':'deephol-data-processed/proofs/human/test'}

In [55]:
# generators
W_HYP = True
BATCH_SIZE = 64
shuffle = False

training_generator = generator.Keras_DataGenerator(data_dir='', dataset='train', 
                                                  w_hyp=W_HYP, batch_size=BATCH_SIZE,
                                                  shuffle=shuffle)
# validation_generator = generator.Keras_DataGenerator(data_dir='', dataset='valid', 
#                                                     w_hyp=W_HYP, batch_size=BATCH_SIZE,
#                                                     shuffle=shuffle)

THIS IS data_dir 
Retrieving data from deephol-data-processed/proofs/human/train/
Generating examples from a set of 376968 examples


In [56]:
training_generator.features_keys_lst[1]

'deephol-data-processed/proofs/human/train/X_train_hyp_2.csv'

## For features
make sure to do both with and without `hypotheses`

In [60]:
# read all
dfs = []
for x in training_generator.features_keys_lst:
    path_x = os.path.join('s3://', BUCKET_NAME, x)
    print(path_x)
    df = pd.read_csv(path_x, header=None)
    dfs.append(df)

s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_1.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_2.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_3.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_4.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_5.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_6.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_7.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_8.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_9.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_10.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_11.csv
s3://sagemaker-cs281/deephol-data-processed/proofs/human/train/X_train_hyp_12.csv
s3://sagemaker-cs281/deep

In [61]:
print([x.shape for x in dfs])
print('\n', len(dfs))

[(5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5027, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026, 3000), (5026

In [62]:
X_train_concat = pd.concat(dfs)
print(X_train_concat.shape)

(376968, 3000)


In [20]:
splits = split_array_batches(X_train)

skipping 48 examples to have equal length batches
partition size: 4096
shape of truncated array:  (122880, 1000)


In [21]:
print([x.shape for x in splits])

[(4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000), (4096, 1000)]


In [22]:
# upload all
for i, x in enumerate(split_array_batches(X_train)):
    key = 'deephol-data-processed/proofs/human/test_new/X_train_{}.csv'.format(i)
    upload_np_to_s3(x, key)
    print(key)

skipping 48 examples to have equal length batches
partition size: 4096
shape of truncated array:  (122880, 1000)
deephol-data-processed/proofs/human/test_new/X_train_0.csv
deephol-data-processed/proofs/human/test_new/X_train_1.csv
deephol-data-processed/proofs/human/test_new/X_train_2.csv
deephol-data-processed/proofs/human/test_new/X_train_3.csv
deephol-data-processed/proofs/human/test_new/X_train_4.csv
deephol-data-processed/proofs/human/test_new/X_train_5.csv
deephol-data-processed/proofs/human/test_new/X_train_6.csv
deephol-data-processed/proofs/human/test_new/X_train_7.csv
deephol-data-processed/proofs/human/test_new/X_train_8.csv
deephol-data-processed/proofs/human/test_new/X_train_9.csv
deephol-data-processed/proofs/human/test_new/X_train_10.csv
deephol-data-processed/proofs/human/test_new/X_train_11.csv
deephol-data-processed/proofs/human/test_new/X_train_12.csv
deephol-data-processed/proofs/human/test_new/X_train_13.csv
deephol-data-processed/proofs/human/test_new/X_train_14.c

In [ ]:
! NOW DO X_train_hyp

## For labels

In [125]:
# read
path_y = os.path.join('s3://', BUCKET_NAME, 
                              os.path.join(data_paths[data_split], 'Y_train.csv'))
print(path_y)
df = pd.read_csv(path_y, header=None)

In [24]:
for i, x in enumerate(split_array_batches(Y_train)):
    key = 'deephol-data-processed/proofs/human/test_new/Y_train_{}.csv'.format(i)
    upload_np_to_s3(x, key)

skipping 48 examples to have equal length batches
partition size: 4096
shape of truncated array:  (122880, 41)


In [139]:
# REVERSE ONE HOT
np.argmax(df.head().values, axis=1)

array([34, 37,  5,  3, 26])

In [23]:
splits = split_array_batches(Y_train)
[x.shape for x in splits]

skipping 48 examples to have equal length batches
partition size: 4096
shape of truncated array:  (122880, 41)


[(4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41)]

[(4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41),
 (4096, 41)]